# MathJsonParser Testing System

This notebook integrates the MathIR parser to run tests on all JSON test files and generate a CSV output with task names and computed answers.

## Requirements

Make sure you have installed the required packages:

```bash
pip install pydantic sympy pandas
```

## How to Run

1. Ensure the `mathir_parser` module is in your Python path
2. Run all cells in this notebook
3. The output CSV file `test_results.csv` will be generated in the current directory

In [ ]:
# Import required modules
import json
import os
import sys
import pandas as pd
from pathlib import Path

# Add the project root to Python path to import mathir_parser
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from mathir_parser.main import MathIR, run_mathir

In [ ]:
# Helper function to load MathIR from JSON file
def load_mathir_from_file(filepath: str) -> MathIR:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return MathIR.model_validate(data)

In [ ]:
# Find all test JSON files in the tests directory
tests_dir = Path('tests')
json_files = list(tests_dir.glob('*.json'))

print(f"Found {len(json_files)} test files:")
for f in json_files:
    print(f"- {f.name}")

In [ ]:
# Process each test file and collect results
results = []

for json_file in json_files:
    try:
        # Load MathIR from file
        ir = load_mathir_from_file(str(json_file))
        
        # Run computation
        computation_results = run_mathir(ir)
        
        # Use filename as task name (without .json extension)
        task_name = json_file.stem
        
        # Convert results to string representation
        # If there are multiple results, join them
        if computation_results:
            answer_str = '; '.join(f"{k}: {str(v)}" for k, v in computation_results.items())
        else:
            answer_str = "No results"
        
        results.append({
            'task': task_name,
            'answer': answer_str
        })
        
        print(f"Processed {task_name}: {answer_str}")
        
    except Exception as e:
        print(f"Error processing {json_file.name}: {str(e)}")
        results.append({
            'task': json_file.stem,
            'answer': f"Error: {str(e)}"
        })

print(f"\nProcessed {len(results)} tasks total")

In [ ]:
# Create DataFrame and save to CSV
df = pd.DataFrame(results)
output_file = 'test_results.csv'
df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")
print("\nPreview of results:")
print(df.head())

In [ ]:
# Display the full CSV content
print("\nFull CSV content:")
with open(output_file, 'r') as f:
    print(f.read())